In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import random
import numpy as np
import google.datalab.storage as storage
from PIL import Image
from scipy import misc
import io

In [2]:
def limit(a, b):
    if a < b:
        return a
    else: return b
    
def generate_cut(image_matrix):
    x,y = image_matrix.shape[:2]
    x_start = random.randint(0, int(x/2))
    y_start = random.randint(0, int(y/2))
    x_end = limit(x_start + int(x/2) + random.randint(0, int(x/16)), x)
    y_end = limit(y_start + int(y/2) + random.randint(0, int(y/16)), y)
    image_cut = image_matrix[x_start:x_end, y_start:y_end]
    return image_cut

def one_hot(x, len_ls):
    a = np.zeros(len_ls, 'int32')
    a[x] = 1
    return a

def gen_file_dict(bucket):
  dict_classes = {}
  my_keys = [_object.key for _object in bucket.objects() if ".jpg" in _object.key]
  not_sources = [not_source for not_source in my_keys if "not_sources" in not_source]
  len_encoding = len(my_keys) - len(not_sources) + 1
  encoding_pos = 0
  
  for key in my_keys:
    if key in not_sources:
      dict_classes[key] = one_hot(len_encoding - 1, len_encoding)
    else:
      dict_classes[key] = one_hot(encoding_pos, len_encoding)
      encoding_pos = encoding_pos + 1
  return dict_classes

sample_bucket = storage.Bucket("copyright")
my_dict = gen_file_dict(sample_bucket)
      

In [3]:
def create_tuples(dict_classes, bucket, mode):
    image_tuples = []
    if mode == "train":
        copies = 200
    elif mode == "test":
        copies = 20
    
    for key in dict_classes.keys():
        if "not_sources" in key and mode not in key:
            continue
        sample_object = bucket.object(key)
        sample_data = sample_object.download()
        stream = io.BytesIO(sample_data)
        my_image = Image.open(stream)
        image_matrix = np.asarray(my_image)
        if len(image_matrix.shape) < 3 or image_matrix.shape[2] != 3:
            print(image_matrix.shape)
            continue
        for i in range(copies):
            input_image = misc.imresize(generate_cut(image_matrix), (40, 60))
            image_tuples.append((input_image, dict_classes[key]))
    return image_tuples

image_tuples_test = create_tuples(my_dict, sample_bucket, "test")
image_tuples_train = create_tuples(my_dict, sample_bucket, "train")

(300, 300)
(278, 300)
(167, 300)
(282, 300)
(164, 300)
(300, 265)
(245, 300)
(300, 298)
(295, 300)


In [12]:
image_tuples_test[23][1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [4]:
num_batches = 100
def create_input(images, num_batches, mode):
    ran_image_tuples = [(a.ravel()/255, np.asarray(b, dtype="float32")) for a,b in images]
    ran_image_lists = list(map(list, zip(*ran_image_tuples)))
    if mode == "train":
        input_batches_list = np.array_split(np.vstack(ran_image_lists[0]), num_batches)
        label_batches_list = np.array_split(np.vstack(ran_image_lists[1]), num_batches)
    elif mode == "test":
        input_batches_list = np.vstack(ran_image_lists[0])
        label_batches_list = np.vstack(ran_image_lists[1])
    else:
        print("te estas inventando el modo o se te ha olvidado")
        return
    return input_batches_list, label_batches_list

random.shuffle(image_tuples_train)
random.shuffle(image_tuples_test)
X_train, Y_train = create_input(image_tuples_train, num_batches, mode="train")
X_test, Y_test = create_input(image_tuples_test, num_batches, mode="test")


In [14]:
len(Y_train[0]), X_train[0].shape

(250, (250, 7200))

In [15]:
#linear model with AdamOptimizer

x_im = 60
y_im = 40
pixel_im = 3
output_dim = len(my_dict["sources/images.jpg"])

x = tf.placeholder(tf.float32, [None, x_im*y_im*pixel_im], name="input")
W = tf.Variable(tf.zeros([x_im*y_im*pixel_im, output_dim]))
b = tf.Variable(tf.zeros([output_dim]))

y = tf.nn.softmax(tf.matmul(x, W) + b, name="output")

y_actual = tf.placeholder(tf.float32, [None, output_dim])

cross_entropy = tf.reduce_mean(
              tf.nn.softmax_cross_entropy_with_logits(labels=y_actual, logits=y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess1 = tf.InteractiveSession()

tf.global_variables_initializer().run()

for j in range(40):
    for i in range(num_batches):
        batch_xs, batch_ys = X_train[i], Y_train[i]
        sess1.run(train_step, feed_dict={x: batch_xs, y_actual: batch_ys})



In [16]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_actual,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess1.run(accuracy, feed_dict={x: X_test, y_actual: Y_test}))
#print(sess1.run(y, feed_dict={x: X_test, y_actual: Y_test}))

0.0795455


In [73]:
Saver = tf.train.Saver()
Saver.save(sess1, "linear_model")

'linear_model'

In [5]:
#convolutional network

x_im = 60
y_im = 40
pixel_im = 3
output_dim = len(my_dict["sources/images.jpg"])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

x = tf.placeholder(tf.float32, [None, x_im*y_im*pixel_im], name="input")
y_actual = tf.placeholder(tf.float32, [None, output_dim])
x_image = tf.reshape(x, [-1, x_im, y_im, pixel_im])

W_conv1 = weight_variable([5, 5, pixel_im, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([15 * 10 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 15*10*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([1024, output_dim])
b_fc2 = bias_variable([output_dim])
y_conv = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name="output")

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_actual, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_actual, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(500):
    if i % 100 == 0:
        sample_batch = random.randint(0, num_batches - 1)
        train_accuracy = accuracy.eval(feed_dict={
                        x: X_train[sample_batch], y_actual: Y_train[sample_batch]})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: X_train[i%num_batches], y_actual: Y_train[i%num_batches]})

print('test accuracy %g' % accuracy.eval(feed_dict={
  x: X_test, y_actual: Y_test}))


step 0, training accuracy 0.004
step 100, training accuracy 0.464
step 200, training accuracy 0.668
step 300, training accuracy 0.876
step 400, training accuracy 0.892
test accuracy 0.903977


In [6]:
Saver = tf.train.Saver()
Saver.save(sess, "convolutional_network")

'convolutional_network'

In [ ]:
for i in range(len(X_test)):
  get_output = tf.argmax(y_conv,1) 
  _input = np.atleast_2d(X_test[i])
  print(sess.run(get_output, feed_dict={x: _input}))

[42]
[65]
[32]
[49]
[3]
[0]
[1]
[72]
[81]
[17]
[54]
[10]
[13]
[11]
[0]
[19]
[37]
[51]
[47]
[81]
[13]
[40]
[32]
[26]
[76]
[64]
[22]
[6]
[65]
[15]
[16]
[80]
[16]
[62]
[30]
[22]
[40]
[31]
[81]
[53]
[57]
[13]
[81]
[49]
[76]
[1]
[76]
[16]
[20]
[57]
[51]
[51]
[55]
[61]
[50]
[81]
[3]
[78]
[36]
[43]
[41]
[81]
[71]
[1]
[51]
[8]
[1]
[81]
[69]
[35]
[9]
[16]
[78]
[9]
[50]
[66]
[13]
[81]
[38]
[61]
[22]
[77]
[38]
[74]
[75]
[20]
[51]
[69]
[25]
[53]
[2]
[36]
[41]
[68]
[22]
[81]
[2]
[11]
[35]
[2]
[81]
[21]
[36]
[46]
[25]
[31]
[77]
[66]
[34]
[12]
[14]
[7]
[59]
[29]
[42]
[21]
[67]
[52]
[35]
[31]
[47]
[40]
[0]
[79]
[61]
[41]
[61]
[56]
[35]
[43]
[76]
[81]
[81]
[81]
[39]
[63]
[66]
[38]
[4]
[33]
[61]
[29]
[43]
[38]
[36]
[54]
[31]
[48]
[65]
[71]
[35]
[15]
[81]
[81]
[41]
[81]
[75]
[0]
[81]
[9]
[19]
[31]
[5]
[45]
[24]
[3]
[71]
[77]
[37]
[43]
[54]
[0]
[12]
[20]
[51]
[25]
[5]
[79]
[1]
[33]
[6]
[14]
[20]
[55]
[36]
[17]
[49]
[81]
[18]
[4]
[81]
[5]
[55]
[60]
[57]
[80]
[69]
[52]
[81]
[40]
[11]
[21]
[39]
[6]
[19]
[31]

In [25]:
print(np.atleast_2d(X_test[1]).shape)

(1, 7200)
